In [ ]:
# === 2D → 3D + Gape (Refined for Phase 1 style) ===
# This cell aligns to the structure used in other Phase 1 notebooks.

# --------------------------------------------------
# Config
# --------------------------------------------------
from pathlib import Path
import os, json, glob, h5py, re, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
from scipy.signal import savgol_filter, find_peaks
from numpy.random import default_rng
import cv2
from scipy.optimize import linear_sum_assignment
import datetime

# Base paths
BASE_DIR = Path("/Users/howardwang/Desktop/Ruten/Evaluation-Metrics_Vishal-main")
SESSION = "2025-05-28_14-12-04_124591"
INFER_ROOT = BASE_DIR / "face/results/n20/inference_raw"
CALIB_JSON = BASE_DIR / "Calibration/calibration.json"
OUTPUT_TAG = "v1"  # increment if re-running

# Cameras to use
CAM_ORDER = [
    "cam-topright",
    "cam-topleft",
    "cam-bottomright",
    "cam-bottomleft",
]

# Input file mapping (predictions.h5 per camera)
PRED_FILES = {
    cam: INFER_ROOT / cam / "predictions.h5" for cam in CAM_ORDER
}

# Output/session directories (non-destructive)
OUTDIR = BASE_DIR / "face/results/n20/triangulated" / f"session_{SESSION}_{OUTPUT_TAG}"
WORKDIR = OUTDIR / "work"
OUTDIR.mkdir(parents=True, exist_ok=True)
WORKDIR.mkdir(parents=True, exist_ok=True)

LOG_PATH = OUTDIR / "pipeline_log.txt"
DEBUG_LOG_PATH = BASE_DIR / ".cursor" / "debug.log"

# Filters / params
FPS_FALLBACK = 120.0
MAX_SPEED_PX = 40          # 2D speed cap (px/frame) for outlier removal
MAX_GAP_2D = 8             # frames
MAX_GAP_3D_SEC = 0.30
REPROJ_STRICT = 10.0       # px
REPROJ_LOOSE = 18.0        # px (only if previous frame was good)
LAG_WINDOW_SEC = 0.5       # fallback lag search +/- window
SAMPLE_FRAMES_ALIGN = 800  # frames for node matching/lag estimation
SAMPLE_NODES_ALIGN = 6

# Suffix for outputs
SUFFIX = OUTPUT_TAG

# --------------------------------------------------
# Utility: logger
# --------------------------------------------------
def log(msg):
    ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    line = f"[{ts}] {msg}"
    print(line)
    with open(LOG_PATH, "a") as f:
        f.write(line + "\n")

def debug_log(hypothesis_id, location, message, data):
    """Write debug log entry in NDJSON format."""
    import json
    entry = {
        "sessionId": "debug-session",
        "runId": "run1",
        "hypothesisId": hypothesis_id,
        "location": location,
        "message": message,
        "data": data,
        "timestamp": int(datetime.datetime.now().timestamp() * 1000)
    }
    try:
        with open(DEBUG_LOG_PATH, "a") as f:
            f.write(json.dumps(entry) + "\n")
    except Exception:
        pass

log("Starting 2D→3D pipeline")
log(f"Session: {SESSION}")
log(f"Outputs: {OUTDIR}")

# --------------------------------------------------
# Aliases for nodes
# --------------------------------------------------
ALIAS = {
    "U": ["u", "upper", "upper_lip", "upperlip", "u_star", "upperlipmarker",
          "toplip", "top_lip", "lip_u", "ulip"],
    "L": ["l", "lower", "lower_lip", "lowerlip", "l_star", "lowerlipmarker",
          "bottomlip", "bot_lip", "lip_l", "llip"],
    "HEAD": ["head", "snout", "nose", "forehead", "ha", "hb", "hc"],
}
MOUTHY = ["lip", "mouth", "jaw", "mandible", "maxilla", "muzzle"]

# camera-key ↔ calibration key mapping
# Calibration uses keys like "cam-topleft.mp4", but we use "cam-topleft" in CAM_ORDER
CAM_KEY = {
    "cam-topleft": "cam-topleft.mp4",
    "cam-topright": "cam-topright.mp4",
    "cam-bottomleft": "cam-bottomleft.mp4",
    "cam-bottomright": "cam-bottomright.mp4",
}

# ------------ FLEXIBLE ALIASES -------------
ALIAS = {
    "U": ["u", "upper", "upper_lip", "upperlip", "u_star", "upperlipmarker",
          "toplip", "top_lip", "lip_u", "ulip"],
    "L": ["l", "lower", "lower_lip", "lowerlip", "l_star", "lowerlipmarker",
          "bottomlip", "bot_lip", "lip_l", "llip"],
    "HEAD": ["head", "snout", "nose", "forehead", "ha", "hb", "hc"]
}
# extra name hints for mouth region (used by auto selector)
MOUTHY = ["lip", "mouth", "jaw", "mandible", "maxilla", "muzzle"]

# ------------ LOAD CALIBRATION -------------
with open(CALIB_JSON, "r") as cf:
    calib = json.load(cf)

# Calibration uses keys like "cam-topleft.mp4"
Ps_raw    = {k: np.array(v) for k, v in calib["P"].items()}
Ks_raw    = {k: np.array(calib["intrinsics"][k]["K"]) for k in Ps_raw}
Dists_raw = {k: np.array(calib["intrinsics"][k]["dist"]) for k in Ps_raw}
IMG_SIZE = tuple(calib.get("img_size", [None, None]))
log(f"[calib] cameras in file: {list(Ps_raw.keys())}, img_size: {IMG_SIZE}")

# Map camera names (from CAM_ORDER) to calibration keys
Ps = {}
Ks = {}
Dists = {}
for cam in CAM_ORDER:
    calib_key = CAM_KEY.get(cam)
    if calib_key and calib_key in Ps_raw:
        Ps[cam] = Ps_raw[calib_key]
        Ks[cam] = Ks_raw[calib_key]
        Dists[cam] = Dists_raw[calib_key]
        # #region agent log
        debug_log("H3", f"calib_mapping_{cam}", "Calibration mapping", {
            "cam": cam, "calib_key": calib_key, "P_shape": list(Ps[cam].shape),
            "K_shape": list(Ks[cam].shape), "dist_shape": list(Dists[cam].shape)
        })
        # #endregion
    else:
        log(f"[warn] calibration missing for {cam} (calib key: {calib_key})")
        # #region agent log
        debug_log("H3", f"calib_missing_{cam}", "Calibration missing", {
            "cam": cam, "calib_key": calib_key, "available_keys": list(Ps_raw.keys())
        })
        # #endregion

if len(Ps) < 2:
    raise RuntimeError(f"Need ≥2 cameras with calibration. Found: {list(Ps.keys())}")

# ------------ HELPERS -------------
def interp_short_nan_runs(y, max_gap):
    """Linear-fill NaN runs up to max_gap samples."""
    y = y.copy(); n = len(y); idx = np.arange(n)
    good = np.isfinite(y)
    if good.sum() < 2: 
        return y
    i = 0
    while i < n:
        if good[i]:
            i += 1; 
            continue
        j = i
        while j < n and not good[j]:
            j += 1
        gap = j - i
        if 0 < gap <= max_gap and i > 0 and j < n and good[i-1] and good[j]:
            y[i:j] = np.interp(idx[i:j], [i-1, j], [y[i-1], y[j]])
            good[i:j] = True
        i = j
    return y

def load_sleap_analysis_h5(path):
    """
    Returns:
        pts [T, N, 2] in pixels (NaNs where missing),
        node_names [N],
        fps (float or None),
        frame_idx [T] (int) or None

    Supports:
      1) Classic SLEAP arrays: (K,T,N,2) / (T,N,2) / variants
      2) "Flat predictions" layout:
         /predictions/{i}.x, /predictions/{i}.y, optional /predictions/{i}.score
         /predictions/frame_idx, /predictions/track
    """
    import h5py, numpy as np, re

    def _extract_fps(f):
        fps = None
        for k in ["video_fps", "videos/fps", "video/fps"]:
            if k in f:
                v = f[k][()]
                fps = float(v[0] if np.size(v) else v); break
        return fps

    def _extract_names(f, N):
        for k in ["node_names","nodes","points/labels","tracks/labels","labels"]:
            if k in f:
                try:
                    nn = [x.decode("utf-8") if isinstance(x, bytes) else str(x)
                          for x in np.array(f[k]).ravel()]
                    return nn
                except Exception:
                    pass
        return [f"node_{i+1}" for i in range(N)]

    def _try_classic(f):
        best = None
        def walk(g, prefix=""):
            nonlocal best
            for k, v in g.items():
                p = f"{prefix}/{k}" if prefix else f"/{k}"
                if isinstance(v, h5py.Group):
                    walk(v, p)
                else:
                    if v.ndim in (3,4) and v.shape[-1] == 2 and np.issubdtype(v.dtype, np.number):
                        s = 0
                        if v.ndim == 4: s += 3
                        if v.shape[-1] == 2: s += 2
                        if max(v.shape) > 50: s += 1
                        if best is None or s > best[0]:
                            best = (s, p, tuple(v.shape))
        walk(f)
        if not best:
            return None
        ds = np.array(f[best[1]])
        # normalize to (K,T,N,2)
        if ds.ndim == 4:
            s = ds.shape
            if s[1] == 2 and s[-1] > 50:          ds = ds.transpose(0,3,2,1)
            elif s[2] == 2 and s[0] > 50:         ds = ds.transpose(3,0,1,2)
            elif s[0] == 2:                        ds = ds.transpose(3,2,1,0)
            elif s[-1] != 2:
                dims = list(s); coord_ax = int(np.where(np.array(dims)==2)[0][0])
                frame_ax = int(np.argmax(dims))
                candidates = [i for i in range(4) if i != coord_ax]
                k_ax = candidates[int(np.argmin([dims[i] for i in candidates]))]
                n_ax = [i for i in range(4) if i not in (coord_ax, frame_ax, k_ax)][0]
                ds = ds.transpose(k_ax, frame_ax, n_ax, coord_ax)
        elif ds.ndim == 3 and ds.shape[-1] == 2:
            ds = ds[None, ...]
        else:
            return None
        K, T, N, _ = ds.shape
        kbest = 0 if K == 1 else int(np.argmax(np.sum(np.isfinite(ds).all(axis=-1), axis=(1,2))))
        pts = ds[kbest].astype(float)
        names = _extract_names(f, N)
        fps = _extract_fps(f)
        # optional scores
        def find_score_key():
            keys = ["point_scores","points_scores","node_scores","point_confidences","scores"]
            for k in keys:
                if k in f: return k
            for grp in f.keys():
                if isinstance(f[grp], h5py.Group):
                    for k in keys:
                        if k in f[grp]: return f"{grp}/{k}"
            return None
        sk = find_score_key()
        if sk is not None:
            sc = np.array(f[sk]).squeeze()
            TT, NN = pts.shape[0], pts.shape[1]
            if sc.ndim == 3 and sc.shape[-1] in (1,2): sc = sc[...,0]
            elif sc.ndim == 2 and sc.shape == (NN,TT): sc = sc.T
            elif sc.ndim == 1 and sc.shape[0] == NN:   sc = np.tile(sc[None,:], (TT,1))
            elif sc.ndim == 1 and sc.shape[0] == TT:   sc = np.tile(sc[:,None], (1,NN))
            elif sc.ndim != 2:                         sc = None
            if sc is not None:
                bad = ~np.isfinite(sc) | (sc <= 0)
                pts[bad] = np.nan
        # no frame_idx here
        return pts, names, fps, None

    def _try_flat_predictions(f):
        if "predictions" not in f: return None
        g = f["predictions"]
        pat = re.compile(r"^(\d+)\.(x|y|score)$")
        nodes = set()
        for k in g.keys():
            m = pat.match(k)
            if m and m.group(2) in ("x","y"):
                nodes.add(int(m.group(1)))
        if not nodes:
            return None
        idxs = sorted(nodes)
        T = g[f"{idxs[0]}.x"].shape[0]
        N = len(idxs)
        pts = np.full((T, N, 2), np.nan, dtype=float)
        for j, nid in enumerate(idxs):
            x = np.array(g[f"{nid}.x"], dtype=float)
            y = np.array(g[f"{nid}.y"], dtype=float)
            pts[:, j, 0] = x
            pts[:, j, 1] = y
            sk = f"{nid}.score"
            if sk in g:
                sc = np.array(g[sk], dtype=float)
                bad = ~np.isfinite(sc) | (sc <= 0)
                pts[bad, j, :] = np.nan
        names = _extract_names(f, N)
        fps = _extract_fps(f)
        frame_idx = None
        if "frame_idx" in g:
            frame_idx = np.array(g["frame_idx"]).astype(int)
        return pts, names, fps, frame_idx

    def _try_sleap_v1_format(f):
        """Handle SLEAP v1 format with frames, instances, pred_points."""
        if "pred_points" not in f or "frames" not in f or "instances" not in f:
            return None
        
        frames = np.array(f["frames"])
        instances = np.array(f["instances"])
        pred_points = np.array(f["pred_points"])
        
        if len(pred_points) == 0 or len(instances) == 0:
            return None
        
        # Determine number of nodes from first instance
        first_inst = instances[0]
        pid_start = int(first_inst["point_id_start"])
        pid_end = int(first_inst["point_id_end"])
        N = pid_end - pid_start
        
        # Get frame indices
        frame_idx = frames["frame_idx"] if "frame_idx" in frames.dtype.names else None
        
        # Build frame -> instances mapping (sorted by score, take best)
        frame_to_instances = {}
        for i, inst in enumerate(instances):
            fid = int(inst["frame_id"])
            if fid not in frame_to_instances:
                frame_to_instances[fid] = []
            frame_to_instances[fid].append((i, float(inst["score"])))
        
        # Sort instances by score (descending) for each frame
        for fid in frame_to_instances:
            frame_to_instances[fid].sort(key=lambda x: x[1], reverse=True)
        
        # Get unique frame IDs and sort
        unique_frame_ids = sorted(frame_to_instances.keys())
        T = len(unique_frame_ids)
        
        # Initialize output array
        pts = np.full((T, N, 2), np.nan, dtype=float)
        
        # Fill in points for each frame (use best instance)
        for t, frame_id in enumerate(unique_frame_ids):
            if not frame_to_instances[frame_id]:
                continue
            # Use the best (first) instance for this frame
            inst_idx, _ = frame_to_instances[frame_id][0]
            inst = instances[inst_idx]
            pid_start = int(inst["point_id_start"])
            pid_end = int(inst["point_id_end"])
            
            if pid_start < len(pred_points) and pid_end <= len(pred_points):
                for node_idx in range(N):
                    pid = pid_start + node_idx
                    if pid < len(pred_points):
                        pt = pred_points[pid]
                        # Check visible flag (complete may be False even for valid points in SLEAP v1)
                        if pt["visible"]:
                            score = float(pt["score"]) if "score" in pt.dtype.names else 1.0
                            if score > 0:
                                pts[t, node_idx, 0] = float(pt["x"])
                                pts[t, node_idx, 1] = float(pt["y"])
        
        # Extract node names from metadata if available
        names = _extract_names(f, N)
        fps = _extract_fps(f)
        
        return pts, names, fps, frame_idx

    with h5py.File(path, "r") as f:
        got = _try_classic(f)
        if got is not None:
            return got
        got = _try_flat_predictions(f)
        if got is not None:
            return got
        got = _try_sleap_v1_format(f)
        if got is not None:
            return got
        raise RuntimeError(f"No usable points dataset found in {path}.")



def find_alias_indices(node_names, alias_dict):
    names_lc = [n.lower() for n in node_names]
    def find_one(cands):
        for i, n in enumerate(names_lc):
            for c in cands:
                if c in n:
                    return i
        return None
    idxU = find_one(alias_dict["U"])
    idxL = find_one(alias_dict["L"])
    idxH = [i for i, n in enumerate(names_lc) for c in alias_dict["HEAD"]
            if c in n and i not in (idxU, idxL)][:3]
    return idxU, idxL, idxH

def name_matches_any(name, substrs):
    n = name.lower()
    return any(s in n for s in substrs)

# ------------ LOAD ALL CAMS -------------
per_cam_pts = {}
per_cam_frameidx = {}
node_names_ref, fps_guess = None, None

for cam in CAM_ORDER:
    pred_path = PRED_FILES[cam]
    if not pred_path.exists():
        log(f"[warn] missing predictions for {cam}: {pred_path}")
        continue
    pts, node_names, fps, fidx = load_sleap_analysis_h5(str(pred_path))
    if node_names_ref is None:
        node_names_ref = node_names
    else:
        if node_names != node_names_ref:
            log(f"[warn] node order differs for {cam}; will realign later")
    per_cam_pts[cam] = pts
    per_cam_frameidx[cam] = fidx  # may be None
    if fps:
        fps_guess = fps
    log(f"[{cam}] loaded {pts.shape} frames, fps≈{fps or 'unknown'} from {pred_path}")
    # #region agent log
    valid_pts = np.isfinite(pts).sum()
    total_pts = pts.size
    debug_log("H1", f"load_after_{cam}", "2D points validity after loading", {
        "cam": cam, "shape": list(pts.shape), "valid_count": int(valid_pts), 
        "total_count": int(total_pts), "valid_fraction": float(valid_pts / total_pts) if total_pts > 0 else 0.0
    })
    # #endregion

if len(per_cam_pts) < 2:
    raise RuntimeError("Need ≥2 cameras with tracks to triangulate.")

# verify calibration keys exist (map cam names to calibration keys)
for cam in per_cam_pts.keys():
    calib_key = CAM_KEY.get(cam)
    if calib_key is None or calib_key not in Ps_raw:
        raise KeyError(f"Calibration missing for camera '{cam}' (calib key: '{calib_key}'). Check CALIB_JSON keys vs CAM_KEY mapping.")
    if cam not in Ps or cam not in Ks or cam not in Dists:
        raise KeyError(f"Calibration mapping failed for camera '{cam}'. Check CAM_KEY mapping.")

FPS = float(fps_guess or 120.0)
print("Using FPS =", FPS)

# ------------ ALIGN CAMERAS IN TIME -------------
cams = list(per_cam_pts.keys())
ref_cam = cams[0]

def shift_with_nans(arr, shift):
    """Positive shift moves data forward (pads front); negative shift pads end."""
    T = arr.shape[0]
    out = np.full_like(arr, np.nan)
    if shift == 0:
        return arr.copy()
    if shift > 0:
        out[shift:] = arr[:T-shift]
    else:
        s = -shift
        out[:T-s] = arr[s:]
    return out

def estimate_lag_by_frameidx(fidx_ref, fidx_cam, max_abs_shift=5_000):
    """Return shift (cam relative to ref) in frames using frame_idx."""
    if fidx_ref is None or fidx_cam is None:
        return None
    # Use median difference of overlapping indices
    # Map frame_idx -> position
    import numpy as np
    ref_map = {int(v): i for i, v in enumerate(fidx_ref)}
    inter = [(ref_map[int(v)], i) for i, v in enumerate(fidx_cam) if int(v) in ref_map]
    if len(inter) < 100:
        return None
    diffs = [ri - ci for (ri, ci) in inter]
    md = int(np.median(diffs))
    if abs(md) > max_abs_shift:
        return None
    return md

# Try frame_idx first
lags = {ref_cam: 0}
for cam in cams[1:]:
    lag = estimate_lag_by_frameidx(per_cam_frameidx.get(ref_cam), per_cam_frameidx.get(cam))
    lags[cam] = lag if lag is not None else 0

# If any None, fall back to a quick search minimizing median pair_err on a few samples
if any(l is None for l in lags.values()):
    # Geometry helpers needed here:
    Rts = {cam: np.linalg.inv(Ks[cam]) @ Ps[cam] for cam in Ps}
    def undist_norm(xy, K, dist):
        return cv2.undistortPoints(xy.reshape(1,1,2).astype(np.float64), K, dist, P=None).reshape(2)
    def tri_2views_norm(Rt1, xn1, Rt2, xn2):
        A = np.stack([xn1[0]*Rt1[2]-Rt1[0], xn1[1]*Rt1[2]-Rt1[1],
                      xn2[0]*Rt2[2]-Rt2[0], xn2[1]*Rt2[2]-Rt2[1]], axis=0)
        _,_,Vt = np.linalg.svd(A); Xh = Vt[-1]; Xh /= Xh[3]; return Xh[:3]
    def undistort_px(xy, cam):
        return cv2.undistortPoints(xy.reshape(1,1,2).astype(np.float64), Ks[cam], Dists[cam], P=Ks[cam]).reshape(2)
    def pair_err(ref_xy, cam_xy, ref_cam, cam):
        xn_ref = undist_norm(ref_xy, Ks[ref_cam], Dists[ref_cam])
        xn_cam = undist_norm(cam_xy, Ks[cam], Dists[cam])
        X = tri_2views_norm(Rts[ref_cam], xn_ref, Rts[cam], xn_cam)
        u_ref = undistort_px(ref_xy, ref_cam)
        u_cam = undistort_px(cam_xy, cam)
        e_ref = np.linalg.norm((Ps[ref_cam] @ np.append(X,1.0))[:2] / (Ps[ref_cam] @ np.append(X,1.0))[2] - u_ref)
        e_cam = np.linalg.norm((Ps[cam]      @ np.append(X,1.0))[:2] / (Ps[cam]      @ np.append(X,1.0))[2] - u_cam)
        return 0.5*(e_ref+e_cam)

    import numpy as np
    Tref, N = per_cam_pts[ref_cam].shape[0], per_cam_pts[ref_cam].shape[1]
    sample_t = np.linspace(0, Tref-1, num=min(400, Tref), dtype=int)
    sample_j = list(range(min(N, 6)))  # 6 nodes max for speed
    lag_window = int(0.5 * FPS)  # search ±0.5 s; increase if needed

    for cam in cams[1:]:
        if lags.get(cam) is not None:
            continue
        best_lag, best_med = 0, np.inf
        for lag in range(-lag_window, lag_window+1, max(1,int(FPS//30))):  # step ~2 frames at 60fps
            errs = []
            for t in sample_t:
                t2 = t - lag
                if t2 < 0 or t2 >= per_cam_pts[cam].shape[0]:
                    continue
                for j in sample_j:
                    xy1 = per_cam_pts[ref_cam][t, j, :]
                    xy2 = per_cam_pts[cam][t2, j, :]
                    if np.all(np.isfinite(xy1)) and np.all(np.isfinite(xy2)):
                        try:
                            errs.append(pair_err(xy1, xy2, ref_cam, cam))
                        except Exception:
                            pass
            if len(errs) > 50:
                med = float(np.median(errs))
                if med < best_med:
                    best_med, best_lag = med, lag
        lags[cam] = best_lag
        print(f"[align] estimated lag {cam} vs {ref_cam}: {best_lag} frames (median reproj err≈{best_med:.2f})")

# Apply lags and re-sync lengths
for cam in cams:
    per_cam_pts[cam] = shift_with_nans(per_cam_pts[cam], lags.get(cam, 0))

lengths = {cam: per_cam_pts[cam].shape[0] for cam in per_cam_pts}
T_common = min(lengths.values())
per_cam_pts = {cam: per_cam_pts[cam][:T_common].copy() for cam in per_cam_pts}
print("[align] applied lags:", lags, "→ new common T =", T_common)

# (then continue with your clean → triangulate steps)


# ------------ CLEAN 2D + SYNC LENGTH -------------
lengths = {cam: per_cam_pts[cam].shape[0] for cam in per_cam_pts}
T_common = min(lengths.values())
per_cam_pts = {cam: per_cam_pts[cam][:T_common].copy() for cam in per_cam_pts}

def mark_outliers_speed(pts, max_px_per_frame=40):
    ok = np.ones((pts.shape[0],), dtype=bool)
    v = np.linalg.norm(np.diff(pts, axis=0), axis=1)
    bad = np.r_[False, v > max_px_per_frame]
    bad |= np.any(~np.isfinite(pts), axis=1)
    ok[bad] = False
    return ok

def fill_short_gaps_2d(pts, max_gap=8):
    out = pts.copy()
    out[:, 0] = interp_short_nan_runs(pts[:, 0], max_gap)
    out[:, 1] = interp_short_nan_runs(pts[:, 1], max_gap)
    return out

for cam in list(per_cam_pts.keys()):
    arr = per_cam_pts[cam]
    for j in range(arr.shape[1]):
        ok = mark_outliers_speed(arr[:, j, :], max_px_per_frame=40)
        arr[~ok, j, :] = np.nan
        arr[:, j, :] = fill_short_gaps_2d(arr[:, j, :], max_gap=8)
    per_cam_pts[cam] = arr
    # #region agent log
    valid_after = np.isfinite(arr).sum()
    total_after = arr.size
    debug_log("H1", f"clean_after_{cam}", "2D points validity after cleaning", {
        "cam": cam, "valid_count": int(valid_after), "total_count": int(total_after),
        "valid_fraction": float(valid_after / total_after) if total_after > 0 else 0.0
    })
    # #endregion

print("[clean] Per-cam lengths:", lengths, "→ using T =", T_common)

# ------------ GEOMETRY / UNDISTORT -------------
Rts = {cam: np.linalg.inv(Ks[cam]) @ Ps[cam] for cam in Ps}

def undist_norm(xy, K, dist):
    return cv2.undistortPoints(xy.reshape(1, 1, 2).astype(np.float64), K, dist, P=None).reshape(2)

def tri_2views_norm(Rt1, xn1, Rt2, xn2):
    A = np.stack([
        xn1[0] * Rt1[2] - Rt1[0],
        xn1[1] * Rt1[2] - Rt1[1],
        xn2[0] * Rt2[2] - Rt2[0],
        xn2[1] * Rt2[2] - Rt2[1],
    ], axis=0)
    _, _, Vt = np.linalg.svd(A)
    Xh = Vt[-1]; Xh /= Xh[3]
    return Xh[:3]

def proj_px(cam, X):
    x = Ps[cam] @ np.append(X, 1.0)
    return (x[:2] / x[2]).astype(float)

def undistort_px(xy, cam):
    return cv2.undistortPoints(xy.reshape(1, 1, 2).astype(np.float64), Ks[cam], Dists[cam], P=Ks[cam]).reshape(2)

def pair_err(ref_xy, cam_xy, ref_cam, cam):
    try:
        xn_ref = undist_norm(ref_xy, Ks[ref_cam], Dists[ref_cam])
        xn_cam = undist_norm(cam_xy, Ks[cam], Dists[cam])
        X = tri_2views_norm(Rts[ref_cam], xn_ref, Rts[cam], xn_cam)
        u_ref = undistort_px(ref_xy, ref_cam)
        u_cam = undistort_px(cam_xy, cam)
        e_ref = np.linalg.norm(proj_px(ref_cam, X) - u_ref)
        e_cam = np.linalg.norm(proj_px(cam, X) - u_cam)
        err = 0.5 * (e_ref + e_cam)
        # #region agent log
        if not np.isfinite(err) or err > 1000:
            debug_log("H2", "pair_err", "High or invalid reprojection error", {
                "ref_cam": ref_cam, "cam": cam, "ref_xy": [float(ref_xy[0]), float(ref_xy[1])],
                "cam_xy": [float(cam_xy[0]), float(cam_xy[1])], "error": float(err) if np.isfinite(err) else None,
                "e_ref": float(e_ref) if np.isfinite(e_ref) else None, "e_cam": float(e_cam) if np.isfinite(e_cam) else None
            })
        # #endregion
        return err
    except Exception as ex:
        # #region agent log
        debug_log("H2", "pair_err_exception", "Exception in pair_err", {
            "ref_cam": ref_cam, "cam": cam, "ref_xy": [float(ref_xy[0]), float(ref_xy[1])] if np.all(np.isfinite(ref_xy)) else None,
            "cam_xy": [float(cam_xy[0]), float(cam_xy[1])] if np.all(np.isfinite(cam_xy)) else None,
            "exception": str(type(ex).__name__), "message": str(ex)
        })
        # #endregion
        raise



# ------------ ALIGN NODE ORDER ACROSS CAMERAS (by geometry) -------------
def estimate_node_permutation_to_ref(ref_cam, cam, sample_frames, min_pairs=60, bad_cost=1e6):
    """
    Returns an array perm of length N where:
      per_cam_pts[cam][:, perm[i], :] ≈ per_cam_pts[ref_cam][:, i, :]
    i.e., perm maps each REF index i to the best matching OLD index in cam.
    """
    T, N = per_cam_pts[ref_cam].shape[0], per_cam_pts[ref_cam].shape[1]
    C = np.full((N, N), bad_cost, dtype=float)  # cost matrix: ref-index i vs cam-index j

    for i in range(N):
        for j in range(N):
            errs = []
            exception_count = 0
            for t in sample_frames:
                if t < 0 or t >= T or t >= per_cam_pts[cam].shape[0]:
                    continue
                xy1 = per_cam_pts[ref_cam][t, i, :]
                xy2 = per_cam_pts[cam][t, j, :]
                if np.all(np.isfinite(xy1)) and np.all(np.isfinite(xy2)):
                    try:
                        errs.append(pair_err(xy1, xy2, ref_cam, cam))
                    except Exception as ex:
                        exception_count += 1
                        pass
            if len(errs) >= min_pairs:
                C[i, j] = float(np.median(errs))
            # #region agent log
            if i == 0 and j == 0:  # Log first pair as sample
                debug_log("H2", f"node_match_{cam}", "Node matching sample", {
                    "ref_cam": ref_cam, "cam": cam, "ref_node": i, "cam_node": j,
                    "valid_pairs": len(errs), "exception_count": exception_count,
                    "median_err": float(np.median(errs)) if len(errs) >= min_pairs else None,
                    "cost": float(C[i, j]) if len(errs) >= min_pairs else None
                })
            # #endregion

    row_ind, col_ind = linear_sum_assignment(C)  # row=ref i, col=cam j
    perm = np.array(col_ind, dtype=int)          # perm[i] = old cam index that matches ref index i
    med_costs = C[row_ind, col_ind]
    return perm, med_costs, C

# choose sample frames for robust stats
T_ref = per_cam_pts[ref_cam].shape[0]
sample_frames = np.linspace(0, T_ref-1, num=min(800, T_ref), dtype=int)

# run for each non-ref cam, then reorder its node axis
for cam in cams:
    if cam == ref_cam:
        continue
    perm, costs, Cmat = estimate_node_permutation_to_ref(ref_cam, cam, sample_frames)
    # simple sanity: if most assigned costs are huge, warn but still apply (can loosen thresholds here)
    n_bad = int(np.sum(~np.isfinite(costs)) + np.sum(costs > 80.0))  # 80 px median reproj err ~ bad
    print(f"[match] {cam} → {ref_cam}: median assigned costs (first 10) = {np.round(costs[:10], 1)}")
    if n_bad > len(costs)//2:
        print(f"[match][warn] many high costs. Check calibration or node consistency.")
    # reorder the camera's node dimension to align with reference indices
    per_cam_pts[cam] = per_cam_pts[cam][:, perm, :]
print("[match] aligned node ordering across cameras using geometric consistency.")



# ------------ AUTO U/L SWAP CHECK (2D) -------------
rng = default_rng(0)
cams = list(per_cam_pts.keys())
ref_cam = cams[0]
T = per_cam_pts[ref_cam].shape[0]
sample_idx = np.arange(0, T, max(1, T // 300))

# try to find U/L by name if present (for swap check only)
idxU_guess, idxL_guess, _ = find_alias_indices(node_names_ref, ALIAS)

swap_votes, total_votes = {c: 0 for c in cams}, {c: 0 for c in cams}
if idxU_guess is not None and idxL_guess is not None:
    for cam in cams:
        if cam == ref_cam: 
            continue
        vs, vt = 0, 0
        for t in sample_idx:
            refU = per_cam_pts[ref_cam][t, idxU_guess, :]
            refL = per_cam_pts[ref_cam][t, idxL_guess, :]
            camU = per_cam_pts[cam][t, idxU_guess, :]
            camL = per_cam_pts[cam][t, idxL_guess, :]
            if not (np.all(np.isfinite(refU)) and np.all(np.isfinite(refL)) and
                    np.all(np.isfinite(camU)) and np.all(np.isfinite(camL))):
                continue
            e0 = pair_err(refU, camU, ref_cam, cam) + pair_err(refL, camL, ref_cam, cam)
            e1 = pair_err(refU, camL, ref_cam, cam) + pair_err(refL, camU, ref_cam, cam)
            vt += 1
            if e1 < e0: 
                vs += 1
        swap_votes[cam] = vs; total_votes[cam] = vt

    cams_to_swap = [c for c in cams if c != ref_cam and total_votes[c] > 0 and (swap_votes[c] / total_votes[c]) > 0.65]
    print("[swap] votes swap/total:", {c: f"{swap_votes[c]}/{total_votes[c]}" for c in cams})
    print("[swap] applying to:", cams_to_swap)
    for cam in cams_to_swap:
        arr = per_cam_pts[cam]
        arr[:, [idxU_guess, idxL_guess], :] = arr[:, [idxL_guess, idxU_guess], :]
        per_cam_pts[cam] = arr
else:
    print("[swap] U/L aliases not found → skipping 2D swap check.")

# ------------ TRIANGULATE ALL NODES -------------
def tri_2views(P1, xy1, P2, xy2):
    A = np.stack([
        xy1[0] * P1[2, :] - P1[0, :],
        xy1[1] * P1[2, :] - P1[1, :],
        xy2[0] * P2[2, :] - P2[0, :],
        xy2[1] * P2[2, :] - P2[1, :],
    ], axis=0)
    _, _, Vt = np.linalg.svd(A)
    X = Vt[-1, :]
    X /= X[3]
    return X[:3]

def reproj_err(P, X, xy):
    x = P @ np.append(X, 1.0)
    x = x[:2] / x[2]
    return float(np.linalg.norm(x - xy))

def best_pair_at_t(j, t):
    best = None  # (err, (c1,c2), X)
    valid_pairs = 0
    exception_count = 0
    for c1, c2 in combinations(cams, 2):
        xy1 = per_cam_pts[c1][t, j, :]
        xy2 = per_cam_pts[c2][t, j, :]
        if not (np.all(np.isfinite(xy1)) and np.all(np.isfinite(xy2))):
            continue
        try:
            u1 = undistort_px(xy1, c1)
            u2 = undistort_px(xy2, c2)
            X  = tri_2views(Ps[c1], u1, Ps[c2], u2)
            e1 = reproj_err(Ps[c1], X, u1)
            e2 = reproj_err(Ps[c2], X, u2)
            e  = 0.5 * (e1 + e2)
            valid_pairs += 1
            if (best is None) or (e < best[0]):
                best = (e, (c1, c2), X)
        except Exception as ex:
            exception_count += 1
            # #region agent log
            if j == 0 and t < 10:  # Log first few as sample
                debug_log("H4", "triangulate_exception", "Exception during triangulation", {
                    "node": j, "frame": t, "c1": c1, "c2": c2,
                    "xy1": [float(xy1[0]), float(xy1[1])], "xy2": [float(xy2[0]), float(xy2[1])],
                    "exception": str(type(ex).__name__), "message": str(ex)
                })
            # #endregion
            continue
    # #region agent log
    if j == 0 and t < 10:  # Log first few frames as sample
        debug_log("H4", "triangulate_sample", "Triangulation attempt", {
            "node": j, "frame": t, "valid_pairs": valid_pairs, "exception_count": exception_count,
            "best_error": float(best[0]) if best is not None else None,
            "best_cams": list(best[1]) if best is not None else None
        })
    # #endregion
    return best

REPROJ_STRICT = 10.0
REPROJ_LOOSE  = 18.0

N = per_cam_pts[ref_cam].shape[1]
X3 = np.full((T, N, 3), np.nan, dtype=float)
accept_counts = np.zeros(N, dtype=int)

for j in range(N):
    prev_good = False
    total_attempts = 0
    accepted = 0
    rejected_strict = 0
    rejected_loose = 0
    for t in range(T):
        bp = best_pair_at_t(j, t)
        if bp is None:
            prev_good = False
            continue
        total_attempts += 1
        e, _, X = bp
        if e <= REPROJ_STRICT or (e <= REPROJ_LOOSE and prev_good):
            X3[t, j, :] = X
            accept_counts[j] += 1
            accepted += 1
            prev_good = True
        else:
            if e > REPROJ_STRICT and not prev_good:
                rejected_strict += 1
            else:
                rejected_loose += 1
            prev_good = False
    # #region agent log
    if j < 3:  # Log first 3 nodes
        debug_log("H4", f"triangulate_node_{j}", "Triangulation summary for node", {
            "node": j, "total_attempts": total_attempts, "accepted": accepted,
            "rejected_strict": rejected_strict, "rejected_loose": rejected_loose,
            "accept_rate": float(accepted / total_attempts) if total_attempts > 0 else 0.0
        })
    # #endregion

# short gap fill (≤0.3 s)
MAX_GAP_3D = int(0.30 * FPS)
for j in range(N):
    for d in range(3):
        X3[:, j, d] = interp_short_nan_runs(X3[:, j, d], MAX_GAP_3D)

cov_by_node = (np.isfinite(X3).all(axis=2)).mean(axis=0)
print("[tri+fill] per-node coverage (first 10):", np.round(cov_by_node[:10], 3))

# ------------ SAVE ALL NODES 3D -------------
all_npz = OUTDIR / f"all_nodes_3d_{SUFFIX}.npz"
all_csv = OUTDIR / f"all_nodes_3d_long_{SUFFIX}.csv"
np.savez_compressed(str(all_npz), X3=X3.astype(np.float32), node_names=np.array(node_names_ref), FPS=np.float32(FPS))
log(f"[OK] wrote all nodes 3D arrays: {all_npz}")

# long-form csv
rows = []
tvec_full = np.arange(T) / FPS
for j, name in enumerate(node_names_ref):
    rows.append(pd.DataFrame({
        "frame": np.arange(T),
        "time_s": tvec_full,
        "node": name,
        "x": X3[:, j, 0], "y": X3[:, j, 1], "z": X3[:, j, 2]
    }))
pd.concat(rows, ignore_index=True).to_csv(all_csv, index=False)
log(f"[OK] wrote all nodes 3D long CSV: {all_csv}")

# ------------ GAPE PAIR SELECTION -------------
def pick_gape_pair(node_names, X3, FPS):
    """Return (idxU, idxL) or (None,None).
       Strategy:
       1) Use aliases if both U and L found.
       2) Else prefer names containing MOUTHY substrings; if none, use all.
       3) Score every pair by robust amplitude × coverage; return best.
    """
    idxU, idxL, _ = find_alias_indices(node_names, ALIAS)
    if idxU is not None and idxL is not None:
        return idxU, idxL

    # candidate index list
    cand = [i for i,n in enumerate(node_names) if name_matches_any(n, MOUTHY)]
    if len(cand) < 2:
        cand = list(range(len(node_names)))  # fallback: all nodes

    cov = np.isfinite(X3).all(axis=2)  # (T,N)
    best_pair, best_score = (None, None), -np.inf
    Tloc = X3.shape[0]

    # Precompute distances efficiently by streaming pairs
    for i, j in combinations(cand, 2):
        good = cov[:, i] & cov[:, j]
        if np.mean(good) < 0.5:
            continue
        d = np.full(Tloc, np.nan, dtype=float)
        dd = X3[:, i, :] - X3[:, j, :]
        g = good.nonzero()[0]
        if g.size > 0:
            d[g] = np.linalg.norm(dd[g, :], axis=1)
        # robust amplitude
        seg = d[np.isfinite(d)]
        if seg.size < int(1.0 * FPS):
            continue
        q10, q90 = np.nanpercentile(seg, [10, 90])
        amp = max(0.0, q90 - q10)
        covg = np.mean(np.isfinite(d))
        score = amp * (0.5 + 0.5 * covg)
        if score > best_score:
            best_score = score
            best_pair = (i, j)

    if best_pair[0] is None:
        return None, None
    # arbitrary order: label the one with higher median Z as "U" (often higher in camera world),
    # but this is arbitrary; downstream treats them symmetrically.
    i, j = best_pair
    zi = np.nanmedian(X3[:, i, 2]); zj = np.nanmedian(X3[:, j, 2])
    return (i, j) if zi >= zj else (j, i)

# ------------ GAPE PIPELINE (only if a pair is found) -------------
idxU, idxL = pick_gape_pair(node_names_ref, X3, FPS)
if idxU is not None and idxL is not None:
    print(f"[gape] using nodes: U={node_names_ref[idxU]}  L={node_names_ref[idxL]}  (indices {idxU},{idxL})")
    U3 = X3[:, idxU, :]
    L3 = X3[:, idxL, :]

    # save the chosen pair
    tri_out_csv = OUTDIR / f"triangulated_UL_3d_{SUFFIX}.csv"
    tri_out_npz = OUTDIR / f"triangulated_UL_3d_{SUFFIX}.npz"
    pd.DataFrame({
        "frame": np.arange(T),
        "time_s": np.arange(T)/FPS,
        "Ux": U3[:,0], "Uy": U3[:,1], "Uz": U3[:,2],
        "Lx": L3[:,0], "Ly": L3[:,1], "Lz": L3[:,2],
    }).to_csv(tri_out_csv, index=False)
    np.savez_compressed(str(tri_out_npz), U3=U3.astype(np.float32), L3=L3.astype(np.float32), FPS=np.float32(FPS))
    log(f"[OK] wrote UL 3D tracks: {tri_out_csv}, {tri_out_npz}")

    # ---- Gape compute (meters) ----
    def odd_leq(n): 
        return max(3, n if n % 2 else n - 1)

    gape = np.full(T, np.nan)
    good3d = np.all(np.isfinite(U3), 1) & np.all(np.isfinite(L3), 1)
    gape[good3d] = np.linalg.norm(U3[good3d] - L3[good3d], axis=1)

    # Hampel + speed cap
    def hampel_1d(x, win, k=3.0):
        x = x.copy(); n = len(x); h = max(3, win//2)
        for i in range(n):
            j0, j1 = max(0, i-h), min(n, i+h+1)
            s = x[j0:j1]; s = s[np.isfinite(s)]
            if s.size < 5 or not np.isfinite(x[i]): 
                continue
            med = np.median(s); mad = np.median(np.abs(s - med)) + 1e-9
            if abs(x[i] - med) > k * 1.4826 * mad:
                x[i] = np.nan
        return x

    gape = hampel_1d(gape, int(0.30 * FPS))
    gape_mm = gape * 1000.0
    dg = np.gradient(gape_mm, 1.0 / FPS)
    gape_mm[np.abs(dg) > 350.0] = np.nan
    gape = interp_short_nan_runs(gape, int(0.25 * FPS))

    PRE_W   = odd_leq(int(0.18 * FPS))
    BASE_W  = odd_leq(int(2.0  * FPS))
    g_pre   = savgol_filter(np.nan_to_num(gape, nan=np.nanmedian(gape)), PRE_W, 2, mode="interp")
    baseline = pd.Series(g_pre).rolling(BASE_W, center=True, 
                                        min_periods=max(3, BASE_W//3)).min().to_numpy()
    gape_zero = np.clip(gape - baseline, 0.0, None)
    gape_zero_mm = gape_zero * 1000.0
    gape_zero_mm[gape_zero_mm > 50.0] = np.nan
    gape_zero_mm = interp_short_nan_runs(gape_zero_mm, int(0.25 * FPS))

    SMOOTH_W = odd_leq(int(0.22 * FPS))
    gape_s_mm = savgol_filter(np.nan_to_num(gape_zero_mm, nan=np.nanmedian(gape_zero_mm)),
                              SMOOTH_W, 3, mode="interp")
    gape_s = gape_s_mm / 1000.0
    acc_s  = savgol_filter(gape_s, SMOOTH_W, 3, deriv=2, delta=1.0/FPS, mode="interp")

    q = np.nanpercentile(gape_s_mm, [0, 25, 50, 75, 95])
    print(f"[debug] gape_s_mm percentiles: min {q[0]:.1f}, p25 {q[1]:.1f}, med {q[2]:.1f}, p75 {q[3]:.1f}, p95 {q[4]:.1f}")

    # best 10s window
    WSEC = 10.0
    win = int(WSEC * FPS)
    stride = max(1, int(0.05 * FPS))
    def robust_amp_mm(xmm):
        q10, q90 = np.nanpercentile(xmm, [10, 90])
        return q90 - q10

    best_f0, best_score = 0, -np.inf
    for f0 in range(0, max(1, T - win + 1), stride):
        seg = gape_s_mm[f0:f0+win]
        cov = np.mean(np.isfinite(seg))
        if cov < 0.85: 
            continue
        sc = robust_amp_mm(seg) * (0.6 + 0.4*cov)
        if sc > best_score:
            best_score, best_f0 = sc, f0

    f0, f1 = best_f0, min(best_f0 + win, T)
    tvec = np.arange(f0, f1) / FPS

    # plots
    fig, ax1 = plt.subplots(figsize=(10, 3.2))
    ax1.plot(tvec, gape_s_mm[f0:f1], label="Gape (smoothed)")
    ax1.set_xlabel("Time (s)"); ax1.set_ylabel("Gape (mm)"); ax1.set_ylim(bottom=0, top=60)
    ax2 = ax1.twinx()
    ax2.plot(tvec, acc_s[f0:f1]*1000.0, color="orange", label="Acceleration")
    ax2.set_ylabel("Acceleration (mm/s²)", color="orange")
    h1,l1 = ax1.get_legend_handles_labels(); h2,l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1+h2, l1+l2, loc="upper right")
    ax1.set_title("Vertical gape (zero-baseline) & acceleration — best 10 s")
    plt.tight_layout()
    plt.savefig(str(OUTDIR / f"gape_10s_overlay_zero_clean_{SUFFIX}.png")); plt.close()

    seg_mm = gape_s_mm[f0:f1]
    pks,_ = find_peaks(seg_mm, distance=max(1, int(0.06*FPS)))
    trs,_ = find_peaks(-seg_mm, distance=max(1, int(0.06*FPS)))
    fig, ax = plt.subplots(figsize=(10, 2.8))
    ax.plot(tvec, seg_mm, lw=1.4)
    ax.scatter(tvec[pks], seg_mm[pks], s=28, marker="o", label="peaks")
    ax.scatter(tvec[trs], seg_mm[trs], s=28, marker="v", label="troughs")
    ax.set_xlabel("Time (s)"); ax.set_ylabel("Gape (mm)"); ax.set_ylim(bottom=0, top=60)
    ax.set_title("Gape — detected cycle landmarks (10 s)")
    ax.legend(); plt.tight_layout()
    plt.savefig(str(OUTDIR / f"gape_10s_cycles_zero_clean_{SUFFIX}.png")); plt.close()

    # 30–40 s panel if available
    if T / FPS > 40.0:
        f0L, f1L = int(30*FPS), int(40*FPS)
        tt = np.arange(f0L, f1L) / FPS
        fig, ax1 = plt.subplots(figsize=(16, 3.2))
        ax1.plot(tt, gape_s_mm[f0L:f1L], label="Gape (smoothed)")
        ax1.set_xlabel("time (s)"); ax1.set_ylabel("gape (mm)")
        ax1.set_ylim(bottom=0, top=60)
        ax2 = ax1.twinx()
        ax2.plot(tt, acc_s[f0L:f1L]*1000.0, color="orange", label="Acceleration")
        ax2.set_ylabel("acceleration (mm/s²)", color="orange")
        h1,l1 = ax1.get_legend_handles_labels(); h2,l2 = ax2.get_legend_handles_labels()
        ax1.legend(h1+h2, l1+l2, loc="upper right")
        ax1.set_title("Vertical gape displacement & acceleration — 30–40 s")
        plt.tight_layout()
        plt.savefig(str(OUTDIR / f"gape_30to40s_overlay_zero_clean_{SUFFIX}.png")); plt.close()

    # export time series
    pd.DataFrame({
        "frame": np.arange(T),
        "time_s": np.arange(T)/FPS,
        "gape_m_raw": gape,
        "gape_m_zero": gape_s_mm/1000.0,
        "gape_mm_zero": gape_s_mm,
        "acc_m_per_s2": acc_s
    }).to_csv(OUTDIR / f"gape_timeseries_from_sleap_zero_{SUFFIX}.csv", index=False)

    log("[DONE] Wrote gape time-series and plots.")
else:
    log("[info] Could not identify a reliable U/L pair → skipped gape. 3D for ALL nodes was saved.")

log("Phase 0 triangulation complete.")

# === End ===


[calib] cameras: ['cam-topleft.mp4', 'cam-topright.mp4', 'cam-bottomleft.mp4', 'cam-bottomright.mp4'] img_size: (600, 600)
[warn] no analysis.h5 for cam-topleft.mp4 (pattern *cam-topleft*.analysis.h5)
[warn] no analysis.h5 for cam-topright.mp4 (pattern *cam-topright*.analysis.h5)
[cam-bottomleft.mp4] loaded (13583, 9, 2) frames, fps≈unknown
[cam-bottomright.mp4] loaded (13335, 9, 2) frames, fps≈unknown
Using FPS = 120.0
[align] applied lags: {'cam-bottomleft.mp4': 0, 'cam-bottomright.mp4': 106} → new common T = 13335
[clean] Per-cam lengths: {'cam-bottomleft.mp4': 13335, 'cam-bottomright.mp4': 13335} → using T = 13335
[match] cam-bottomright.mp4 → cam-bottomleft.mp4: median assigned costs (first 10) = [69.5 62.1  8.4  8.9 55.4 21.4 31.  45.  46.3]
[match] aligned node ordering across cameras using geometric consistency.
[swap] U/L aliases not found → skipping 2D swap check.
[tri+fill] per-node coverage (first 10): [0.    0.    0.92  0.804 0.    0.073 0.02  0.002 0.002]
[OK] wrote all n

In [ ]:
# Phase 0: 2D → 3D Triangulation (batch runner)
# Aligned with Phase 1 style; outputs written to face/results/n20/pipeline/phase0/

from pathlib import Path
import os, json, h5py, re, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
from scipy.signal import savgol_filter, find_peaks
from numpy.random import default_rng
import cv2
from scipy.optimize import linear_sum_assignment

# ------------------ CONFIG ------------------
BASE_DIR = Path("/Users/howardwang/Desktop/Ruten/Evaluation-Metrics_Vishal-main")
SESSION = "2025-05-28_14-12-04_124591"
OUTPUT_TAG = "v1"
CALIB_JSON = BASE_DIR / "Calibration/calibration.json"
WORKDIR = BASE_DIR / "face/results/n20/pipeline/phase0" / f"session_{SESSION}_{OUTPUT_TAG}" / "work"
OUTDIR = WORKDIR.parent
LOG_PATH = OUTDIR / "pipeline_log.txt"

CAM_ORDER = ["cam-topright", "cam-topleft", "cam-bottomright", "cam-bottomleft"]
PRED_FILES = {cam: WORKDIR / f"{cam}.analysis.h5" for cam in CAM_ORDER}

FPS_FALLBACK = 120.0
MAX_SPEED_PX = 40
MAX_GAP_2D = 8
MAX_GAP_3D_SEC = 0.30
REPROJ_STRICT = 10.0
REPROJ_LOOSE = 18.0
LAG_WINDOW_SEC = 0.5
SAMPLE_FRAMES_ALIGN = 800
SAMPLE_NODES_ALIGN = 6
SUFFIX = OUTPUT_TAG

# ------------------ LOGGER ------------------
def log(msg):
    ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    line = f"[{ts}] {msg}"
    print(line)
    with open(LOG_PATH, "a") as f:
        f.write(line + "\n")

WORKDIR.mkdir(parents=True, exist_ok=True)
log("Phase 0 triangulation start")
log(f"Session={SESSION}, OUTDIR={OUTDIR}")

# ------------------ ALIASES ------------------
ALIAS = {
    "U": ["u","upper","upper_lip","upperlip","u_star","upperlipmarker","toplip","top_lip","lip_u","ulip"],
    "L": ["l","lower","lower_lip","lowerlip","l_star","lowerlipmarker","bottomlip","bot_lip","lip_l","llip"],
    "HEAD": ["head","snout","nose","forehead","ha","hb","hc"],
}
MOUTHY = ["lip","mouth","jaw","mandible","maxilla","muzzle"]

# ------------------ HELPERS ------------------
def interp_short_nan_runs(y, max_gap):
    y = y.copy(); n = len(y); idx = np.arange(n)
    good = np.isfinite(y)
    if good.sum() < 2:
        return y
    i = 0
    while i < n:
        if good[i]:
            i += 1; continue
        j = i
        while j < n and not good[j]:
            j += 1
        gap = j - i
        if 0 < gap <= max_gap and i > 0 and j < n and good[i-1] and good[j]:
            y[i:j] = np.interp(idx[i:j], [i-1, j], [y[i-1], y[j]])
            good[i:j] = True
        i = j
    return y


def load_sleap_h5(path):
    def _extract_fps(f):
        fps = None
        for k in ["video_fps","videos/fps","video/fps"]:
            if k in f:
                v = f[k][()]; fps = float(v[0] if np.size(v) else v); break
        return fps
    def _extract_names(f, N):
        for k in ["node_names","nodes","points/labels","tracks/labels","labels"]:
            if k in f:
                try:
                    nn = [x.decode("utf-8") if isinstance(x, bytes) else str(x) for x in np.array(f[k]).ravel()]
                    return nn
                except Exception:
                    pass
        return [f"node_{i+1}" for i in range(N)]
    def _try_classic(f):
        best = None
        def walk(g, prefix=""):
            nonlocal best
            for k, v in g.items():
                p = f"{prefix}/{k}" if prefix else f"/{k}"
                if isinstance(v, h5py.Group):
                    walk(v, p)
                else:
                    if v.ndim in (3,4) and v.shape[-1] == 2 and np.issubdtype(v.dtype, np.number):
                        s = 0
                        if v.ndim == 4: s += 3
                        if v.shape[-1] == 2: s += 2
                        if max(v.shape) > 50: s += 1
                        if best is None or s > best[0]:
                            best = (s, p, tuple(v.shape))
        walk(f)
        if not best:
            return None
        ds = np.array(f[best[1]])
        if ds.ndim == 4:
            s = ds.shape
            if s[1] == 2 and s[-1] > 50:          ds = ds.transpose(0,3,2,1)
            elif s[2] == 2 and s[0] > 50:         ds = ds.transpose(3,0,1,2)
            elif s[0] == 2:                        ds = ds.transpose(3,2,1,0)
            elif s[-1] != 2:
                dims = list(s); coord_ax = int(np.where(np.array(dims)==2)[0][0])
                frame_ax = int(np.argmax(dims))
                candidates = [i for i in range(4) if i != coord_ax]
                k_ax = candidates[int(np.argmin([dims[i] for i in candidates]))]
                n_ax = [i for i in range(4) if i not in (coord_ax, frame_ax, k_ax)][0]
                ds = ds.transpose(k_ax, frame_ax, n_ax, coord_ax)
        elif ds.ndim == 3 and ds.shape[-1] == 2:
            ds = ds[None, ...]
        else:
            return None
        K, T, N, _ = ds.shape
        kbest = 0 if K == 1 else int(np.argmax(np.sum(np.isfinite(ds).all(axis=-1), axis=(1,2))))
        pts = ds[kbest].astype(float)
        names = _extract_names(f, N)
        fps = _extract_fps(f)
        def find_score_key():
            keys = ["point_scores","points_scores","node_scores","point_confidences","scores"]
            for k in keys:
                if k in f: return k
            for grp in f.keys():
                if isinstance(f[grp], h5py.Group):
                    for k in keys:
                        if k in f[grp]: return f"{grp}/{k}"
            return None
        sk = find_score_key()
        if sk is not None:
            sc = np.array(f[sk]).squeeze()
            TT, NN = pts.shape[0], pts.shape[1]
            if sc.ndim == 3 and sc.shape[-1] in (1,2): sc = sc[...,0]
            elif sc.ndim == 2 and sc.shape == (NN,TT): sc = sc.T
            elif sc.ndim == 1 and sc.shape[0] == NN:   sc = np.tile(sc[None,:], (TT,1))
            elif sc.ndim == 1 and sc.shape[0] == TT:   sc = np.tile(sc[:,None], (1,NN))
            elif sc.ndim != 2:                         sc = None
            if sc is not None:
                bad = ~np.isfinite(sc) | (sc <= 0)
                pts[bad] = np.nan
        return pts, names, fps, None
    def _try_flat_predictions(f):
        if "predictions" not in f: return None
        g = f["predictions"]
        pat = re.compile(r"^(\d+)\.(x|y|score)$")
        nodes = set()
        for k in g.keys():
            m = pat.match(k)
            if m and m.group(2) in ("x","y"):
                nodes.add(int(m.group(1)))
        if not nodes:
            return None
        idxs = sorted(nodes)
        T = g[f"{idxs[0]}.x"].shape[0]
        N = len(idxs)
        pts = np.full((T, N, 2), np.nan, dtype=float)
        for j, nid in enumerate(idxs):
            x = np.array(g[f"{nid}.x"], dtype=float)
            y = np.array(g[f"{nid}.y"], dtype=float)
            pts[:, j, 0] = x
            pts[:, j, 1] = y
            sk = f"{nid}.score"
            if sk in g:
                sc = np.array(g[sk], dtype=float)
                bad = ~np.isfinite(sc) | (sc <= 0)
                pts[bad, j, :] = np.nan
        names = _extract_names(f, N)
        fps = _extract_fps(f)
        frame_idx = None
        if "frame_idx" in g:
            frame_idx = np.array(g["frame_idx"]).astype(int)
        return pts, names, fps, frame_idx
    with h5py.File(path, "r") as f:
        got = _try_classic(f)
        if got is not None:
            return got
        got = _try_flat_predictions(f)
        if got is not None:
            return got
        raise RuntimeError(f"No usable points dataset found in {path}.")


def find_alias_indices(node_names, alias_dict):
    names_lc = [n.lower() for n in node_names]
    def find_one(cands):
        for i, n in enumerate(names_lc):
            for c in cands:
                if c in n:
                    return i
        return None
    idxU = find_one(alias_dict["U"])
    idxL = find_one(alias_dict["L"])
    idxH = [i for i, n in enumerate(names_lc) for c in alias_dict["HEAD"] if c in n and i not in (idxU, idxL)][:3]
    return idxU, idxL, idxH

def name_matches_any(name, substrs):
    n = name.lower(); return any(s in n for s in substrs)

# ------------------ LOAD CALIBRATION ------------------
with open(CALIB_JSON, "r") as cf:
    calib = json.load(cf)
Ps    = {k: np.array(v) for k, v in calib["P"].items()}
Ks    = {k: np.array(calib["intrinsics"][k]["K"]) for k in Ps}
Dists = {k: np.array(calib["intrinsics"][k]["dist"]) for k in Ps}
IMG_SIZE = tuple(calib.get("img_size", [None, None]))
log(f"[calib] cameras: {list(Ps.keys())}, img_size: {IMG_SIZE}")

# ------------------ LOAD PREDICTIONS ------------------
per_cam_pts = {}
per_cam_frameidx = {}
node_names_ref, fps_guess = None, None
for cam in CAM_ORDER:
    path = PRED_FILES[cam]
    if not path.exists():
        log(f"[warn] missing predictions for {cam}: {path}")
        continue
    pts, node_names, fps, fidx = load_sleap_h5(path)
    if node_names_ref is None:
        node_names_ref = node_names
    else:
        if node_names != node_names_ref:
            log(f"[warn] node order differs for {cam}; will realign later")
    per_cam_pts[cam] = pts
    per_cam_frameidx[cam] = fidx
    if fps:
        fps_guess = fps
    log(f"[{cam}] loaded {pts.shape} frames, fps≈{fps or 'unknown'} from {path}")

if len(per_cam_pts) < 2:
    raise RuntimeError("Need ≥2 cameras with tracks to triangulate.")
for cam in per_cam_pts.keys():
    if cam not in Ps or cam not in Ks or cam not in Dists:
        raise KeyError(f"Calibration missing for camera key '{cam}'.")

FPS = float(fps_guess or FPS_FALLBACK)
log(f"Using FPS = {FPS}")

# ------------------ TIME ALIGNMENT ------------------
def shift_with_nans(arr, shift):
    T = arr.shape[0]; out = np.full_like(arr, np.nan)
    if shift == 0: return arr.copy()
    if shift > 0: out[shift:] = arr[:T-shift]
    else: out[:T+shift] = arr[-shift:]
    return out

def estimate_lag_by_frameidx(fidx_ref, fidx_cam, max_abs_shift=5000):
    if fidx_ref is None or fidx_cam is None:
        return None
    ref_map = {int(v): i for i, v in enumerate(fidx_ref)}
    inter = [(ref_map.get(int(v)), i) for i, v in enumerate(fidx_cam) if int(v) in ref_map]
    inter = [(ri, ci) for (ri, ci) in inter if ri is not None]
    if len(inter) < 100:
        return None
    diffs = [ri - ci for (ri, ci) in inter]
    md = int(np.median(diffs))
    if abs(md) > max_abs_shift:
        return None
    return md

lags = {}
ref_cam = CAM_ORDER[0]
lag_window = int(LAG_WINDOW_SEC * FPS)
sample_t_count = min(SAMPLE_FRAMES_ALIGN, per_cam_pts[ref_cam].shape[0])
sample_t = np.linspace(0, per_cam_pts[ref_cam].shape[0]-1, num=sample_t_count, dtype=int)

Rts = {cam: np.linalg.inv(Ks[cam]) @ Ps[cam] for cam in Ps}

def undist_norm(xy, K, dist):
    return cv2.undistortPoints(xy.reshape(1,1,2).astype(np.float64), K, dist, P=None).reshape(2)

def tri_2views_norm(Rt1, xn1, Rt2, xn2):
    A = np.stack([xn1[0]*Rt1[2]-Rt1[0], xn1[1]*Rt1[2]-Rt1[1], xn2[0]*Rt2[2]-Rt2[0], xn2[1]*Rt2[2]-Rt2[1]], axis=0)
    _,_,Vt = np.linalg.svd(A); Xh = Vt[-1]; Xh /= Xh[3]; return Xh[:3]

def undistort_px(xy, cam):
    return cv2.undistortPoints(xy.reshape(1,1,2).astype(np.float64), Ks[cam], Dists[cam], P=Ks[cam]).reshape(2)

def pair_err(ref_xy, cam_xy, ref_cam, cam):
    xn_ref = undist_norm(ref_xy, Ks[ref_cam], Dists[ref_cam]); xn_cam = undist_norm(cam_xy, Ks[cam], Dists[cam])
    X = tri_2views_norm(Rts[ref_cam], xn_ref, Rts[cam], xn_cam)
    u_ref = undistort_px(ref_xy, ref_cam); u_cam = undistort_px(cam_xy, cam)
    e_ref = np.linalg.norm((Ps[ref_cam] @ np.append(X,1.0))[:2] / (Ps[ref_cam] @ np.append(X,1.0))[2] - u_ref)
    e_cam = np.linalg.norm((Ps[cam]      @ np.append(X,1.0))[:2] / (Ps[cam]      @ np.append(X,1.0))[2] - u_cam)
    return 0.5*(e_ref+e_cam)

lags[ref_cam] = 0
for cam in CAM_ORDER[1:]:
    if cam not in per_cam_pts:
        continue
    lag = estimate_lag_by_frameidx(per_cam_frameidx.get(ref_cam), per_cam_frameidx.get(cam))
    if lag is not None:
        lags[cam] = lag; continue
    best_lag, best_med = 0, np.inf
    sample_j = list(range(min(SAMPLE_NODES_ALIGN, per_cam_pts[ref_cam].shape[1])))
    for lag in range(-lag_window, lag_window+1, max(1,int(FPS//30))):
        errs = []
        for t in sample_t:
            t2 = t - lag
            if t2 < 0 or t2 >= per_cam_pts[cam].shape[0]:
                continue
            for j in sample_j:
                xy1 = per_cam_pts[ref_cam][t, j, :]
                xy2 = per_cam_pts[cam][t2, j, :]
                if np.all(np.isfinite(xy1)) and np.all(np.isfinite(xy2)):
                    try:
                        errs.append(pair_err(xy1, xy2, ref_cam, cam))
                    except Exception:
                        pass
        if len(errs) > 50:
            med = float(np.median(errs))
            if med < best_med:
                best_med, best_lag = med, lag
    lags[cam] = best_lag
    log(f"[align] estimated lag {cam} vs {ref_cam}: {best_lag} frames (median reproj err≈{best_med:.2f})")

for cam in CAM_ORDER:
    if cam not in per_cam_pts:
        continue
    per_cam_pts[cam] = shift_with_nans(per_cam_pts[cam], lags.get(cam, 0))

lengths = {cam: per_cam_pts[cam].shape[0] for cam in per_cam_pts}
T_common = min(lengths.values())
per_cam_pts = {cam: per_cam_pts[cam][:T_common].copy() for cam in per_cam_pts}
log(f"[align] applied lags: {lags} → T = {T_common}")

# ------------------ CLEAN 2D ------------------
def mark_outliers_speed(pts, max_px_per_frame=MAX_SPEED_PX):
    ok = np.ones((pts.shape[0],), dtype=bool)
    v = np.linalg.norm(np.diff(pts, axis=0), axis=1)
    bad = np.r_[False, v > max_px_per_frame]
    bad |= np.any(~np.isfinite(pts), axis=1)
    ok[bad] = False
    return ok

def fill_short_gaps_2d(pts, max_gap=MAX_GAP_2D):
    out = pts.copy()
    out[:, 0] = interp_short_nan_runs(pts[:, 0], max_gap)
    out[:, 1] = interp_short_nan_runs(pts[:, 1], max_gap)
    return out

for cam in list(per_cam_pts.keys()):
    arr = per_cam_pts[cam]
    for j in range(arr.shape[1]):
        ok = mark_outliers_speed(arr[:, j, :])
        arr[~ok, j, :] = np.nan
        arr[:, j, :] = fill_short_gaps_2d(arr[:, j, :])
    per_cam_pts[cam] = arr
log(f"[clean] per-cam lengths: {lengths} → T = {T_common}")

# ------------------ MATCH NODES ------------------
ref_cam = CAM_ORDER[0]
T = per_cam_pts[ref_cam].shape[0]
sample_frames = np.linspace(0, T-1, num=min(SAMPLE_FRAMES_ALIGN, T), dtype=int)

def estimate_node_permutation_to_ref(ref_cam, cam, sample_frames, min_pairs=60, bad_cost=1e6):
    Tloc, N = per_cam_pts[ref_cam].shape[0], per_cam_pts[ref_cam].shape[1]
    C = np.full((N, N), bad_cost, dtype=float)
    for i in range(N):
        for j in range(N):
            errs = []
            for t in sample_frames:
                if t < 0 or t >= Tloc or t >= per_cam_pts[cam].shape[0]:
                    continue
                xy1 = per_cam_pts[ref_cam][t, i, :]
                xy2 = per_cam_pts[cam][t, j, :]
                if np.all(np.isfinite(xy1)) and np.all(np.isfinite(xy2)):
                    try:
                        errs.append(pair_err(xy1, xy2, ref_cam, cam))
                    except Exception:
                        pass
            if len(errs) >= min_pairs:
                C[i, j] = float(np.median(errs))
    row_ind, col_ind = linear_sum_assignment(C)
    perm = np.array(col_ind, dtype=int)
    med_costs = C[row_ind, col_ind]
    return perm, med_costs, C

for cam in CAM_ORDER:
    if cam == ref_cam or cam not in per_cam_pts:
        continue
    perm, costs, _ = estimate_node_permutation_to_ref(ref_cam, cam, sample_frames)
    bad_count = int(np.sum(~np.isfinite(costs)) + np.sum(costs > 80.0))
    log(f"[match] {cam} → {ref_cam}: median costs (first 10) {np.round(costs[:10],1)}")
    if bad_count > len(costs)//2:
        log("[match][warn] many high costs; check calibration or node consistency.")
    per_cam_pts[cam] = per_cam_pts[cam][:, perm, :]
log("[match] node ordering aligned across cameras")

# ------------------ OPTIONAL U/L SWAP ------------------
rng = default_rng(0)
sample_idx = np.arange(0, T, max(1, T // 300))
idxU_guess, idxL_guess, _ = find_alias_indices(node_names_ref, ALIAS)
swap_votes, total_votes = {c: 0 for c in per_cam_pts}, {c: 0 for c in per_cam_pts}
if idxU_guess is not None and idxL_guess is not None:
    for cam in per_cam_pts:
        if cam == ref_cam:
            continue
        vs, vt = 0, 0
        for t in sample_idx:
            refU = per_cam_pts[ref_cam][t, idxU_guess, :]
            refL = per_cam_pts[ref_cam][t, idxL_guess, :]
            camU = per_cam_pts[cam][t, idxU_guess, :]
            camL = per_cam_pts[cam][t, idxL_guess, :]
            if not (np.all(np.isfinite(refU)) and np.all(np.isfinite(refL)) and np.all(np.isfinite(camU)) and np.all(np.isfinite(camL))):
                continue
            e0 = pair_err(refU, camU, ref_cam, cam) + pair_err(refL, camL, ref_cam, cam)
            e1 = pair_err(refU, camL, ref_cam, cam) + pair_err(refL, camU, ref_cam, cam)
            vt += 1
            if e1 < e0:
                vs += 1
        swap_votes[cam] = vs; total_votes[cam] = vt
    cams_to_swap = [c for c in per_cam_pts if c != ref_cam and total_votes[c] > 0 and (swap_votes[c] / total_votes[c]) > 0.65]
    log(f"[swap] votes swap/total: { {c: f'{swap_votes[c]}/{total_votes[c]}' for c in per_cam_pts} }")
    if cams_to_swap:
        log(f"[swap] applying to: {cams_to_swap}")
        for cam in cams_to_swap:
            arr = per_cam_pts[cam]
            arr[:, [idxU_guess, idxL_guess], :] = arr[:, [idxL_guess, idxU_guess], :]
            per_cam_pts[cam] = arr
else:
    log("[swap] U/L aliases not found → skipping swap check")

# ------------------ TRIANGULATION ------------------
N = per_cam_pts[ref_cam].shape[1]
X3 = np.full((T, N, 3), np.nan, dtype=float)
accept_counts = np.zeros(N, dtype=int)

def tri_2views(P1, xy1, P2, xy2):
    A = np.stack([
        xy1[0] * P1[2, :] - P1[0, :],
        xy1[1] * P1[2, :] - P1[1, :],
        xy2[0] * P2[2, :] - P2[0, :],
        xy2[1] * P2[2, :] - P2[1, :],
    ], axis=0)
    _, _, Vt = np.linalg.svd(A)
    X = Vt[-1, :]
    X /= X[3]
    return X[:3]

def reproj_err(P, X, xy):
    x = P @ np.append(X, 1.0)
    x = x[:2] / x[2]
    return float(np.linalg.norm(x - xy))

def best_pair_at_t(j, t):
    best = None
    for c1, c2 in combinations(per_cam_pts.keys(), 2):
        xy1 = per_cam_pts[c1][t, j, :]
        xy2 = per_cam_pts[c2][t, j, :]
        if not (np.all(np.isfinite(xy1)) and np.all(np.isfinite(xy2))):
            continue
        u1 = undistort_px(xy1, c1)
        u2 = undistort_px(xy2, c2)
        X  = tri_2views(Ps[c1], u1, Ps[c2], u2)
        e1 = reproj_err(Ps[c1], X, u1)
        e2 = reproj_err(Ps[c2], X, u2)
        e  = 0.5 * (e1 + e2)
        if (best is None) or (e < best[0]):
            best = (e, (c1, c2), X)
    return best

for j in range(N):
    prev_good = False
    for t in range(T):
        bp = best_pair_at_t(j, t)
        if bp is None:
            prev_good = False
            continue
        e, _, X = bp
        if e <= REPROJ_STRICT or (e <= REPROJ_LOOSE and prev_good):
            X3[t, j, :] = X
            accept_counts[j] += 1
            prev_good = True
        else:
            prev_good = False

MAX_GAP_3D = int(MAX_GAP_3D_SEC * FPS)
for j in range(N):
    for d in range(3):
        X3[:, j, d] = interp_short_nan_runs(X3[:, j, d], MAX_GAP_3D)

cov_by_node = (np.isfinite(X3).all(axis=2)).mean(axis=0)
log(f"[tri+fill] per-node coverage (first 10): {np.round(cov_by_node[:10], 3)}")

# ------------------ SAVE 3D OUTPUTS ------------------
all_npz = OUTDIR / f"all_nodes_3d_{SUFFIX}.npz"
all_csv = OUTDIR / f"all_nodes_3d_long_{SUFFIX}.csv"
np.savez_compressed(all_npz, X3=X3.astype(np.float32), node_names=np.array(node_names_ref), FPS=np.float32(FPS))
rows = []
tvec_full = np.arange(T) / FPS
for j, name in enumerate(node_names_ref):
    rows.append(pd.DataFrame({
        "frame": np.arange(T),
        "time_s": tvec_full,
        "node": name,
        "x": X3[:, j, 0], "y": X3[:, j, 1], "z": X3[:, j, 2],
    }))
pd.concat(rows, ignore_index=True).to_csv(all_csv, index=False)
log(f"[OK] wrote 3D outputs: {all_npz}, {all_csv}")

# ------------------ GAPE (optional) ------------------
def pick_gape_pair(node_names, X3, FPS):
    idxU, idxL, _ = find_alias_indices(node_names, ALIAS)
    if idxU is not None and idxL is not None:
        return idxU, idxL
    cand = [i for i,n in enumerate(node_names) if name_matches_any(n, MOUTHY)]
    if len(cand) < 2:
        cand = list(range(len(node_names)))
    cov = np.isfinite(X3).all(axis=2)
    best_pair, best_score = (None, None), -np.inf
    Tloc = X3.shape[0]
    for i, j in combinations(cand, 2):
        good = cov[:, i] & cov[:, j]
        if np.mean(good) < 0.5:
            continue
        d = np.full(Tloc, np.nan, dtype=float)
        dd = X3[:, i, :] - X3[:, j, :]
        g = good.nonzero()[0]
        if g.size > 0:
            d[g] = np.linalg.norm(dd[g, :], axis=1)
        seg = d[np.isfinite(d)]
        if seg.size < int(1.0 * FPS):
            continue
        q10, q90 = np.nanpercentile(seg, [10, 90])
        amp = max(0.0, q90 - q10)
        covg = np.mean(np.isfinite(d))
        score = amp * (0.5 + 0.5 * covg)
        if score > best_score:
            best_score = score; best_pair = (i, j)
    if best_pair[0] is None:
        return None, None
    i, j = best_pair; zi = np.nanmedian(X3[:, i, 2]); zj = np.nanmedian(X3[:, j, 2])
    return (i, j) if zi >= zj else (j, i)

idxU, idxL = pick_gape_pair(node_names_ref, X3, FPS)
if idxU is not None and idxL is not None:
    log(f"[gape] using nodes: U={node_names_ref[idxU]}  L={node_names_ref[idxL]}  (idx {idxU},{idxL})")
    U3 = X3[:, idxU, :]; L3 = X3[:, idxL, :]
    tri_out_csv = OUTDIR / f"triangulated_UL_3d_{SUFFIX}.csv"
    tri_out_npz = OUTDIR / f"triangulated_UL_3d_{SUFFIX}.npz"
    pd.DataFrame({
        "frame": np.arange(T),
        "time_s": np.arange(T)/FPS,
        "Ux": U3[:,0], "Uy": U3[:,1], "Uz": U3[:,2],
        "Lx": L3[:,0], "Ly": L3[:,1], "Lz": L3[:,2],
    }).to_csv(tri_out_csv, index=False)
    np.savez_compressed(tri_out_npz, U3=U3.astype(np.float32), L3=L3.astype(np.float32), FPS=np.float32(FPS))
    log(f"[OK] wrote UL 3D tracks: {tri_out_csv}")

    def odd_leq(n): return max(3, n if n % 2 else n - 1)

    gape = np.full(T, np.nan)
    good3d = np.all(np.isfinite(U3), 1) & np.all(np.isfinite(L3), 1)
    gape[good3d] = np.linalg.norm(U3[good3d] - L3[good3d], axis=1)

    def hampel_1d(x, win, k=3.0):
        x = x.copy(); n = len(x); h = max(3, win//2)
        for i in range(n):
            j0, j1 = max(0, i-h), min(n, i+h+1)
            s = x[j0:j1]; s = s[np.isfinite(s)]
            if s.size < 5 or not np.isfinite(x[i]):
                continue
            med = np.median(s); mad = np.median(np.abs(s - med)) + 1e-9
            if abs(x[i] - med) > k * 1.4826 * mad:
                x[i] = np.nan
        return x

    gape = hampel_1d(gape, int(0.30 * FPS))
    gape_mm = gape * 1000.0
    dg = np.gradient(gape_mm, 1.0 / FPS)
    gape_mm[np.abs(dg) > 350.0] = np.nan
    gape = interp_short_nan_runs(gape, int(0.25 * FPS))

    PRE_W = odd_leq(int(0.18 * FPS))
    BASE_W = odd_leq(int(2.0  * FPS))
    g_pre = savgol_filter(np.nan_to_num(gape, nan=np.nanmedian(gape)), PRE_W, 2, mode="interp")
    baseline = pd.Series(g_pre).rolling(BASE_W, center=True, min_periods=max(3, BASE_W//3)).min().to_numpy()
    gape_zero = np.clip(gape - baseline, 0.0, None)
    gape_zero_mm = gape_zero * 1000.0
    gape_zero_mm[gape_zero_mm > 50.0] = np.nan
    gape_zero_mm = interp_short_nan_runs(gape_zero_mm, int(0.25 * FPS))

    SMOOTH_W = odd_leq(int(0.22 * FPS))
    gape_s_mm = savgol_filter(np.nan_to_num(gape_zero_mm, nan=np.nanmedian(gape_zero_mm)), SMOOTH_W, 3, mode="interp")
    gape_s = gape_s_mm / 1000.0
    acc_s  = savgol_filter(gape_s, SMOOTH_W, 3, deriv=2, delta=1.0/FPS, mode="interp")

    q = np.nanpercentile(gape_s_mm, [0, 25, 50, 75, 95])
    log(f"[gape] percentiles mm: {q}")

    WSEC = 10.0; win = int(WSEC * FPS); stride = max(1, int(0.05 * FPS))
    def robust_amp_mm(xmm):
        q10, q90 = np.nanpercentile(xmm, [10, 90]); return q90 - q10

    best_f0, best_score = 0, -np.inf
    for f0 in range(0, max(1, T - win + 1), stride):
        seg = gape_s_mm[f0:f0+win]; cov = np.mean(np.isfinite(seg))
        if cov < 0.85: continue
        sc = robust_amp_mm(seg) * (0.6 + 0.4*cov)
        if sc > best_score:
            best_score, best_f0 = sc, f0

    f0, f1 = best_f0, min(best_f0 + win, T); tvec = np.arange(f0, f1) / FPS

    fig, ax1 = plt.subplots(figsize=(10, 3.2))
    ax1.plot(tvec, gape_s_mm[f0:f1], label="Gape (smoothed)")
    ax1.set_xlabel("Time (s)"); ax1.set_ylabel("Gape (mm)"); ax1.set_ylim(bottom=0, top=60)
    ax2 = ax1.twinx(); ax2.plot(tvec, acc_s[f0:f1]*1000.0, color="orange", label="Acceleration")
    ax2.set_ylabel("Acceleration (mm/s²)", color="orange")
    h1,l1 = ax1.get_legend_handles_labels(); h2,l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1+h2, l1+l2, loc="upper right")
    ax1.set_title("Vertical gape (zero-baseline) & acceleration — best 10 s")
    plt.tight_layout(); plt.savefig(OUTDIR / f"gape_10s_overlay_zero_clean_{SUFFIX}.png"); plt.close()

    seg_mm = gape_s_mm[f0:f1]; pks,_ = find_peaks(seg_mm, distance=max(1, int(0.06*FPS)))
    trs,_ = find_peaks(-seg_mm, distance=max(1, int(0.06*FPS)))
    fig, ax = plt.subplots(figsize=(10, 2.8))
    ax.plot(tvec, seg_mm, lw=1.4)
    ax.scatter(tvec[pks], seg_mm[pks], s=28, marker="o", label="peaks")
    ax.scatter(tvec[trs], seg_mm[trs], s=28, marker="v", label="troughs")
    ax.set_xlabel("Time (s)"); ax.set_ylabel("Gape (mm)"); ax.set_ylim(bottom=0, top=60)
    ax.set_title("Gape — detected cycle landmarks (10 s)")
    ax.legend(); plt.tight_layout(); plt.savefig(OUTDIR / f"gape_10s_cycles_zero_clean_{SUFFIX}.png"); plt.close()

    if T / FPS > 40.0:
        f0L, f1L = int(30*FPS), int(40*FPS); tt = np.arange(f0L, f1L) / FPS
        fig, ax1 = plt.subplots(figsize=(16, 3.2))
        ax1.plot(tt, gape_s_mm[f0L:f1L], label="Gape (smoothed)"); ax1.set_xlabel("time (s)"); ax1.set_ylabel("gape (mm)"); ax1.set_ylim(bottom=0, top=60)
        ax2 = ax1.twinx(); ax2.plot(tt, acc_s[f0L:f1L]*1000.0, color="orange", label="Acceleration"); ax2.set_ylabel("acceleration (mm/s²)", color="orange")
        h1,l1 = ax1.get_legend_handles_labels(); h2,l2 = ax2.get_legend_handles_labels(); ax1.legend(h1+h2, l1+l2, loc="upper right")
        ax1.set_title("Vertical gape displacement & acceleration — 30–40 s")
        plt.tight_layout(); plt.savefig(OUTDIR / f"gape_30to40s_overlay_zero_clean_{SUFFIX}.png"); plt.close()

    pd.DataFrame({
        "frame": np.arange(T),
        "time_s": np.arange(T)/FPS,
        "gape_m_raw": gape,
        "gape_m_zero": gape_s_mm/1000.0,
        "gape_mm_zero": gape_s_mm,
        "acc_m_per_s2": acc_s,
    }).to_csv(OUTDIR / f"gape_timeseries_from_sleap_zero_{SUFFIX}.csv", index=False)
    log("[DONE] Wrote gape time-series and plots.")
else:
    log("[info] Could not identify a reliable U/L pair → skipped gape. 3D for all nodes was saved.")

log("Phase 0 triangulation complete.")

